# Thesis

## Install Library

In [ ]:
!pip install roboflow
!pip install ultralytics

## Import Library

In [ ]:
import time
import os
import csv
import shutil
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics.data.annotator import auto_annotate

## Download Dataset

In [ ]:
# Dataset Object Detection
rf = Roboflow(api_key="MQdx0fMQ8FiQPaS1VHRH")
project = rf.workspace("abiya-thesis").project("plant-pathology-2021-object-detection-j1jvh")
version = project.version(9)
dataset = version.download("yolov11", location="dataset/object_detection")

In [ ]:
# Dataset Mask Lesi
rf = Roboflow(api_key="MQdx0fMQ8FiQPaS1VHRH")
project = rf.workspace("abiya-thesis").project("plant-pathology-2021-instance-segmentation-h5pim")
version = project.version(4)
dataset = version.download("yolov11", location="dataset/mask_lesi")

In [ ]:
# Dataset Mask Lesi and Leaf
rf = Roboflow(api_key="MQdx0fMQ8FiQPaS1VHRH")
project = rf.workspace("abiya-thesis").project("plant-pathology-2021-instance-segmentation-h5pim")
version = project.version(5)
dataset = version.download("yolov11", location="dataset/mask_lesi_and_leaf")

In [ ]:
# Dataset Leaf Class Only
rf = Roboflow(api_key="MQdx0fMQ8FiQPaS1VHRH")
project = rf.workspace("abiya-thesis").project("plant-pathology-2021-object-detection-j1jvh")
version = project.version(11)
dataset = version.download("yolov11", location="dataset/object_detection_leaf_class_only")        

## Google Drive

In [ ]:
GoogleDrive = True
if GoogleDrive:
    from google.colab import drive
    drive.mount('/content/gdrive')
    def copy_to_google_drive(source_folder, destination_folder):
        !cp -r /content/{source_folder} /content/gdrive/MyDrive/{destination_folder}

## Global Function

In [ ]:
def measure_and_save_time(start_time, output_path):
    elapsed = time.time() - start_time
    h, rem = divmod(elapsed, 3600)
    m, s = divmod(rem, 60)
    formatted = f"{int(h)}h {int(m)}m {int(s)}s"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as f:
        f.write(formatted)

## Training Object Detection

In [ ]:
# Variabel Global
models = ['yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt']
sizes = ['nano', 'small', 'medium']
epochs = 100
batch = -1 # atau -1 untuk limitasi gpu 60%
data_path_objectDetection = "dataset/object_detection"
project_base_objectDetection = "results/object_detection"

In [ ]:
for model_name, size in zip(models, sizes):
    print(f"Training {size} model...")
    model = YOLO(model_name)

    start_time = time.time()
    training = model.train(
        data=f"{data_path_objectDetection}/data.yaml", 
        epochs=epochs, 
        imgsz=640, 
        batch=batch, 
        project=f"{project_base_objectDetection}/training", 
        name=f"{size}",
        exist_ok=True
    )

    measure_and_save_time(start_time, f"{project_base_objectDetection}/training/summary/time/{size}-train-time.txt")
    csv_filename = f"{project_base_objectDetection}/training/summary/{size}-training-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(training.to_csv())

    validation = model.val(
        data=f"{data_path_objectDetection}/data.yaml", 
        imgsz=640,
        project=f"{project_base_objectDetection}/validation",
        name=f"{size}",
        exist_ok=True,
        split="test"
    )

    csv_filename = f"{project_base_objectDetection}/validation/summary/{size}-validation-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(validation.to_csv())

    print(f"Finished training and validating {size} model.")

In [ ]:
for size in sizes:
    print(f"Running prediction for {size} model...")

    model = YOLO(f"results/object_detection/training/{size}/weights/best.pt")

    # Define the output directory for the current model size
    output_dir = f"results/object_detection/predict/{size}"
    os.makedirs(output_dir, exist_ok=True)

    results = model(
        source=f"{data_path_objectDetection}/test/images", 
        exist_ok=True,
        stream=True
    )

    for i, result in enumerate(results):
        original_filename = os.path.basename(result.path)
        save_path = os.path.join(output_dir, original_filename)
        result.save(filename=save_path)

    print(f"Finished predicting for {size} model.")

In [ ]:
%matplotlib inline
for size in sizes:
    df = pd.read_csv(f"{project_base_objectDetection}/training/{size}/results.csv")

    plt.figure(figsize=(10, 6))

    # Plot mAP50 dan mAP50-95
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")

    # Tambahkan label dan judul
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title(f"Model Performance - Model {size}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Simpan plot
    graph_output_dir = f"{project_base_objectDetection}/graph"
    os.makedirs(graph_output_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_output_dir, f"mAP_comparison_{size}.png"))
    plt.show()

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/object_detection", "dataset/object_detection")
    copy_to_google_drive("results/object_detection", "results/object_detection")

## Semi Auto Annotate Mask Daun

In [ ]:
splits = ["train","valid",'test']
sam2_model = "sam2.1_b.pt"
data_path_maskDaun = "dataset/mask_daun"
project_base_maskDaun = "results/mask_daun"
best_objectDetection_model_size = "medium"
best_objectDetection_model_path = f"{project_base_objectDetection}/training/{best_objectDetection_model_size}/weights/best.pt"

In [ ]:
# Pindahkan original images ke mask daun
for split in splits:
    shutil.copytree(src=f"{data_path_objectDetection}/{split}/images", dst=f"{data_path_maskDaun}/{split}/images", dirs_exist_ok=True)

shutil.copy(src=f"{data_path_objectDetection}/data.yaml", dst=f"{data_path_maskDaun}/data.yaml")

In [ ]:
for split in splits:
    auto_annotate(data=f"{data_path_maskDaun}/{split}/images/", det_model=best_objectDetection_model_path, sam_model=sam2_model, output_dir=f"{data_path_maskDaun}/{split}/labels/")

## Training Mask Daun

In [ ]:
# Variabel Global
models = ['yolo11n-seg.pt', 'yolo11s-seg.pt', 'yolo11m-seg.pt']
sizes = ['nano', 'small', 'medium']
epochs = 100
batch = -1 # atau -1 untuk limitasi gpu 60%
data_path_maskDaun = "dataset/mask_daun"
project_base_maskDaun = "results/mask_daun"

In [ ]:
for model_name, size in zip(models, sizes):
    print(f"Training {size} model...")
    model = YOLO(model_name)

    start_time = time.time()
    training = model.train(
        data=f"{data_path_maskDaun}/data.yaml", 
        epochs=epochs, 
        imgsz=640, 
        batch=batch, 
        project=f"{project_base_maskDaun}/training", 
        name=f"{size}",
        exist_ok=True
    )

    measure_and_save_time(start_time, f"{project_base_maskDaun}/training/summary/time/{size}-train-time.txt")
    csv_filename = f"{project_base_maskDaun}/training/summary/{size}-training-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(training.to_csv())

    validation = model.val(
        data=f"{data_path_maskDaun}/data.yaml", 
        imgsz=640,
        project=f"{project_base_maskDaun}/validation",
        name=f"{size}",
        exist_ok=True,
        split="test"
    )

    csv_filename = f"{project_base_maskDaun}/validation/summary/{size}-validation-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(validation.to_csv())

    print(f"Finished training and validating {size} model.")

In [ ]:
for size in sizes:
    print(f"Running prediction for {size} model...")

    model = YOLO(f"results/mask_daun/training/{size}/weights/best.pt")

    # Define the output directory for the current model size
    output_dir = f"results/mask_daun/predict/{size}"
    os.makedirs(output_dir, exist_ok=True)

    results = model(
        source=f"{data_path_maskDaun}/test/images", 
        exist_ok=True,
        stream=True
    )

    for i, result in enumerate(results):
        original_filename = os.path.basename(result.path)
        save_path = os.path.join(output_dir, original_filename)
        result.save(filename=save_path)

    print(f"Finished predicting for {size} model.")

In [ ]:
%matplotlib inline
for size in sizes:
    df = pd.read_csv(f"{project_base_maskDaun}/training/{size}/results.csv")

    plt.figure(figsize=(10, 6))

    # Plot mAP50 dan mAP50-95
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")

    # Tambahkan label dan judul
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title(f"Model Performance - Model {size}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Simpan plot
    graph_output_dir = f"{project_base_maskDaun}/graph"
    os.makedirs(graph_output_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_output_dir, f"mAP_comparison_{size}.png"))
    plt.show()

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/mask_daun", "dataset/mask_daun")
    copy_to_google_drive("results/mask_daun", "results/mask_daun")

## Training Mask Lesi

In [ ]:
# Variabel Global
models = ['yolo11n-seg.pt', 'yolo11s-seg.pt', 'yolo11m-seg.pt']
sizes = ['nano', 'small', 'medium']
epochs = 100
batch = -1 # atau -1 untuk limitasi gpu 60%
data_path_maskLesi = "dataset/mask_lesi"
project_base_maskLesi = "results/mask_lesi"

In [ ]:
for model_name, size in zip(models, sizes):
    print(f"Training {size} model...")
    model = YOLO(model_name)

    start_time = time.time()
    training = model.train(
        data=f"{data_path_maskLesi}/data.yaml", 
        epochs=epochs, 
        imgsz=640, 
        batch=batch, 
        project=f"{project_base_maskLesi}/training", 
        name=f"{size}",
        exist_ok=True
    )

    measure_and_save_time(start_time, f"{project_base_maskLesi}/training/summary/time/{size}-train-time.txt")
    csv_filename = f"{project_base_maskLesi}/training/summary/{size}-training-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(training.to_csv())

    validation = model.val(
        data=f"{data_path_maskLesi}/data.yaml", 
        imgsz=640,
        project=f"{project_base_maskLesi}/validation",
        name=f"{size}",
        exist_ok=True,
        split="test"
    )

    csv_filename = f"{project_base_maskLesi}/validation/summary/{size}-validation-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(validation.to_csv())

    print(f"Finished training and validating {size} model.")

In [ ]:
for size in sizes:
    print(f"Running prediction for {size} model...")

    model = YOLO(f"results/mask_lesi/training/{size}/weights/best.pt")

    # Define the output directory for the current model size
    output_dir = f"results/mask_lesi/predict/{size}"
    os.makedirs(output_dir, exist_ok=True)

    results = model(
        source=f"{data_path_maskLesi}/test/images", # Kedepannya ubah menjadi /test jika sudah ada test yang di anotasi
        exist_ok=True,
        stream=True
    )

    for i, result in enumerate(results):
        original_filename = os.path.basename(result.path)
        save_path = os.path.join(output_dir, original_filename)
        result.save(filename=save_path)

    print(f"Finished predicting for {size} model.")

In [ ]:
%matplotlib inline
for size in sizes:
    df = pd.read_csv(f"{project_base_maskLesi}/training/{size}/results.csv")

    plt.figure(figsize=(10, 6))

    # Plot mAP50 dan mAP50-95
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")

    # Tambahkan label dan judul
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title(f"Model Performance - Model {size}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Simpan plot
    graph_output_dir = f"{project_base_maskLesi}/graph"
    os.makedirs(graph_output_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_output_dir, f"mAP_comparison_{size}.png"))
    plt.show()

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/mask_lesi", "dataset/mask_lesi")
    copy_to_google_drive("results/mask_lesi", "results/mask_lesi")

## Training Mask Lesi dan Daun

In [ ]:
# Variabel Global
models = ['yolo11n-seg.pt', 'yolo11s-seg.pt', 'yolo11m-seg.pt']
sizes = ['nano', 'small', 'medium']
epochs = 100
batch = -1 # atau -1 untuk limitasi gpu 60%
data_path_maskLesi_and_maskLeaf = "dataset/mask_lesi_and_leaf"
project_base_maskLesi_and_maskLeaf = "results/mask_lesi_and_leaf"

In [ ]:
for model_name, size in zip(models, sizes):
    print(f"Training {size} model...")
    model = YOLO(model_name)

    start_time = time.time()
    training = model.train(
        data=f"{data_path_maskLesi_and_maskLeaf}/data.yaml", 
        epochs=epochs, 
        imgsz=640, 
        batch=batch, 
        project=f"{project_base_maskLesi_and_maskLeaf}/training", 
        name=f"{size}",
        exist_ok=True
    )

    measure_and_save_time(start_time, f"{project_base_maskLesi_and_maskLeaf}/training/summary/time/{size}-train-time.txt")
    csv_filename = f"{project_base_maskLesi_and_maskLeaf}/training/summary/{size}-training-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(training.to_csv())

    validation = model.val(
        data=f"{data_path_maskLesi_and_maskLeaf}/data.yaml", 
        imgsz=640,
        project=f"{project_base_maskLesi}/validation",
        name=f"{size}",
        exist_ok=True,
        split="test"
    )

    csv_filename = f"{project_base_maskLesi}/validation/summary/{size}-validation-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(validation.to_csv())

    print(f"Finished training and validating {size} model.")

In [ ]:
for size in sizes:
    print(f"Running prediction for {size} model...")

    model = YOLO(f"results/mask_lesi/training/{size}/weights/best.pt")

    # Define the output directory for the current model size
    output_dir = f"results/mask_lesi/predict/{size}"
    os.makedirs(output_dir, exist_ok=True)

    results = model(
        source=f"{data_path_maskLesi_and_maskLeaf}/test/images",
        exist_ok=True,
        stream=True
    )

    for i, result in enumerate(results):
        original_filename = os.path.basename(result.path)
        save_path = os.path.join(output_dir, original_filename)
        result.save(filename=save_path)

    print(f"Finished predicting for {size} model.")

In [ ]:
%matplotlib inline
for size in sizes:
    df = pd.read_csv(f"{project_base_maskLesi_and_maskLeaf}/training/{size}/results.csv")

    plt.figure(figsize=(10, 6))

    # Plot mAP50 dan mAP50-95
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")

    # Tambahkan label dan judul
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title(f"Model Performance - Model {size}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Simpan plot
    graph_output_dir = f"{project_base_maskLesi_and_maskLeaf}/graph"
    os.makedirs(graph_output_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_output_dir, f"mAP_comparison_{size}.png"))
    plt.show()

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/mask_lesi_and_leaf", "dataset/mask_lesi_and_leaf")
    copy_to_google_drive("results/mask_lesi_and_leaf", "results/mask_lesi_and_leaf")

## Object Detection Class Leaf Only

In [ ]:
# Variabel Global
models = ['yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt']
sizes = ['nano', 'small', 'medium']
epochs = 100
batch = -1 # atau -1 untuk limitasi gpu 60%
data_path_objectDetection_leaf_class_only = "dataset/object_detection_leaf_class_only"
project_base_objectDetection_leaf_class_only = "results/object_detection_leaf_class_only"

In [ ]:
for model_name, size in zip(models, sizes):
    print(f"Training {size} model...")
    model = YOLO(model_name)

    training = model.train(
        data=f"{data_path_objectDetection_leaf_class_only}/data.yaml", 
        epochs=epochs, 
        imgsz=640, 
        batch=batch, 
        project=f"{project_base_objectDetection_leaf_class_only}/training", 
        name=f"{size}",
        exist_ok=True
    )

    csv_filename = f"{project_base_objectDetection_leaf_class_only}/training/summary/{size}-training-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(training.to_csv())

    validation = model.val(
        data=f"{data_path_objectDetection_leaf_class_only}/data.yaml", 
        imgsz=640,
        project=f"{project_base_objectDetection_leaf_class_only}/validation",
        name=f"{size}",
        exist_ok=True,
        split="test"
    )

    csv_filename = f"{project_base_objectDetection_leaf_class_only}/validation/summary/{size}-validation-metrics.csv"
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    with open(csv_filename, "w") as f:
        f.write(validation.to_csv())

    print(f"Finished training and validating {size} model.")

In [ ]:
for size in sizes:
    print(f"Running prediction for {size} model...")

    model = YOLO(f"results/object_detection_leaf_class_only/training/{size}/weights/best.pt")

    # Define the output directory for the current model size
    output_dir = f"results/object_detection_leaf_class_only/predict/{size}"
    os.makedirs(output_dir, exist_ok=True)

    results = model(
        source=f"{data_path_objectDetection_leaf_class_only}/test/images", 
        exist_ok=True,
        stream=True
    )

    for i, result in enumerate(results):
        original_filename = os.path.basename(result.path)
        save_path = os.path.join(output_dir, original_filename)
        result.save(filename=save_path)

    print(f"Finished predicting for {size} model.")

In [ ]:
%matplotlib inline
for size in sizes:
    df = pd.read_csv(f"{project_base_objectDetection_leaf_class_only}/training/{size}/results.csv")

    plt.figure(figsize=(10, 6))

    # Plot mAP50 dan mAP50-95
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")

    # Tambahkan label dan judul
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title(f"Model Performance - Model {size}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Simpan plot
    graph_output_dir = f"{project_base_objectDetection_leaf_class_only}/graph"
    os.makedirs(graph_output_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_output_dir, f"mAP_comparison_{size}.png"))
    plt.show()

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/object_detection_leaf_class_only", "dataset/object_detection_leaf_class_only")
    copy_to_google_drive("results/object_detection_leaf_class_only", "results/object_detection_leaf_class_only")

## Fine Tune For Detect Healthy, Rust, and Scab

In [ ]:
# Path sumber dataset YOLO
image_dir = Path("dataset/object_detection/train/images")
label_dir = Path("dataset/object_detection/train/labels")

# Path target few-shot
few_shot_base = Path("dataset/few_shot")

# Mapping class ID ke nama
class_map = {0: "frog-eye-leaf-spot", 1: "healthy", 2: "rust"}

# Buat index gambar per class berdasarkan nama file
class_to_files = defaultdict(list)
for img_file in sorted(image_dir.glob("*.jpg")):
    filename_lower = img_file.name.lower()
    if "frog-eye-leaf-spot" in filename_lower:
        cls = 0
    elif "healthy" in filename_lower:
        cls = 1
    elif "rust" in filename_lower:
        cls = 2
    else:
        continue  # Skip jika tidak dikenali

    label_file = label_dir / (img_file.stem + ".txt")
    if label_file.exists():
        class_to_files[cls].append((img_file, label_file))

# Generate few-shot data
for n in [5] + list(range(10, 51, 5)):
    target_image_dir = few_shot_base / f"{n}-shot" / "images"
    target_label_dir = few_shot_base / f"{n}-shot" / "labels"
    target_image_dir.mkdir(parents=True, exist_ok=True)
    target_label_dir.mkdir(parents=True, exist_ok=True)

    for cls in class_map:
        files = class_to_files[cls][:n]  # ambil berurutan
        for img_path, label_path in files:
            shutil.copy(img_path, target_image_dir / img_path.name)
            shutil.copy(label_path, target_label_dir / label_path.name)

# Copy data.yaml
shutil.copy("dataset/object_detection/data.yaml", "dataset/few_shot/data.yaml")

In [ ]:
# Variabel Global
models = {"nano": "results/object_detection_leaf_class_only/training/nano/weights/best.pt",
          "small": "results/object_detection_leaf_class_only/training/small/weights/best.pt",
          "medium": "results/object_detection_leaf_class_only/training/medium/weights/best.pt"}
sizes = ["nano", "small", "medium"]
shots = [5] + list(range(10, 51, 5))
size_to_shots = {
    "nano":   shots,
    "small":  shots,
    "medium": shots
}
epochs = 50
batch = -1 # atau -1 untuk limitasi gpu 60%
data_path_few_shot = "dataset/few_shot"
project_base_few_shot = "results/few_shot"

In [ ]:
for size in sizes:
    for shot in size_to_shots[size]:
        # 1. Load original YAML
        data_yaml_path = f"{data_path_few_shot}/data.yaml"
        with open(data_yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)

        # 2. Ubah path train sesuai shot
        data_yaml['train'] = f"../{shot}-shot/images"

        # 3. Simpan YAML baru ke file sementara
        temp_yaml_path = f"{data_path_few_shot}/data-shot.yaml"
        with open(temp_yaml_path, 'w') as f:
            yaml.dump(data_yaml, f, sort_keys=False, default_flow_style=True)

        model = YOLO(models[size])
        start_time = time.time()
        training = model.train(
            data = temp_yaml_path,
            epochs = epochs,
            imgsz=640,
            batch=batch,
            project=f"{project_base_few_shot}/training",
            name=f"{shot}-shot-{size}",
            exist_ok=True,
            # patience=25,
            # freeze=8
        )

        csv_filename = f"{project_base_few_shot}/training/summary/{shot}-shot-{size}-training-metrics.csv"
        os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
        with open(csv_filename, "w") as f:
            f.write(training.to_csv())

        validation = model.val(
            data = temp_yaml_path,
            imgsz=640,
            project=f"{project_base_few_shot}/validation",
            name=f"{shot}-shot-{size}",
            exist_ok=True,
            split="test"
        )

        csv_filename = f"{project_base_few_shot}/validation/summary/{shot}-shot-{size}-validation-metrics.csv"
        os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
        with open(csv_filename, "w") as f:
            f.write(validation.to_csv())

In [ ]:
%matplotlib inline
for shot in shots:
    df = pd.read_csv(f"{project_base_few_shot}/training/{shot}-shot-{size}/results.csv")

    plt.figure(figsize=(10, 6))

    # Plot mAP50 dan mAP50-95
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")

    # Tambahkan label dan judul
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title(f"Model Performance - {size} | {shot}-shot")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Simpan plot
    graph_output_dir = f"{project_base_few_shot}/graph"
    os.makedirs(graph_output_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_output_dir, f"mAP_comparison_{shot}-shot-{size}.png"))
    plt.show()

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/few_shot", "dataset/few_shot")
    copy_to_google_drive("results/few_shot", "results/few_shot")

## Perbandingan Beberapa k-query Set vs n-shot

In [ ]:
# Path sumber dataset YOLO
image_dir = Path("dataset/object_detection/train/images")
label_dir = Path("dataset/object_detection/train/labels")

# Path target support-set
support_set_base = Path("dataset/support_set")

# Mapping class ID ke nama
class_map = {0: "frog-eye-leaf-spot", 1: "healthy", 2: "rust"}

# Buat index gambar per class berdasarkan nama file
class_to_files = defaultdict(list)
for img_file in sorted(image_dir.glob("*.jpg")):
    filename_lower = img_file.name.lower()
    if "frog-eye-leaf-spot" in filename_lower:
        cls = 0
    elif "healthy" in filename_lower:
        cls = 1
    elif "rust" in filename_lower:
        cls = 2
    else:
        continue  # Skip jika tidak dikenali

    label_file = label_dir / (img_file.stem + ".txt")
    if label_file.exists():
        class_to_files[cls].append((img_file, label_file))

# Generate support-set data
for n in [5] + list(range(10, 51, 5)):
    target_image_dir = support_set_base / f"{n}-images" / "images"
    target_label_dir = support_set_base / f"{n}-images" / "labels"
    target_image_dir.mkdir(parents=True, exist_ok=True)
    target_label_dir.mkdir(parents=True, exist_ok=True)

    for cls in class_map:
        files = class_to_files[cls][-n:]  # ambil n gambar terakhir
        for img_path, label_path in files:
            shutil.copy(img_path, target_image_dir / img_path.name)
            shutil.copy(label_path, target_label_dir / label_path.name)

# Copy data.yaml
shutil.copy("dataset/object_detection/data.yaml", "dataset/support_set/data.yaml")

In [ ]:
# ======================================================
# Variabel Global (disesuaikan dengan training sebelumnya)
# ======================================================
sizes = ["nano", "small", "medium"]
shots = [5] + list(range(10, 51, 5))
project_base_few_shot = "results/few_shot"
support_set_base = "dataset/support_set"

# ======================================================
# Setup data.yaml original (support_set)
# ======================================================
data_yaml_original_path = f"{support_set_base}/data.yaml"
temp_data_yaml_dir = os.path.dirname(data_yaml_original_path)
os.makedirs(temp_data_yaml_dir, exist_ok=True)

# ======================================================
# Dictionary hasil evaluasi
# ======================================================
all_map50_scores = {size: {} for size in sizes}

# ======================================================
# Loop evaluasi setiap model few-shot
# ======================================================
for size in sizes:
    for shot in shots:
        print(f"\nValidasi {size} model ({shot}-shot) ...")

        # Path model hasil training few-shot
        model_path = f"{project_base_few_shot}/training/{shot}-shot-{size}/weights/best.pt"
        model = YOLO(model_path)

        # Load YAML original
        with open(data_yaml_original_path, 'r') as f:
            data_yaml = yaml.safe_load(f)

        # Set path test → support-set tertentu
        data_yaml['test'] = f"../{shot}-images/images"
        data_yaml['val']  = f"../{shot}-images/images"
        data_yaml['train'] = f"../{shot}-images/images"

        # Simpan yaml sementara
        temp_yaml_path = os.path.join(temp_data_yaml_dir, f"data_eval_{shot}-shot-{size}.yaml")
        with open(temp_yaml_path, 'w') as f:
            yaml.dump(data_yaml, f, sort_keys=False, default_flow_style=False)

        # Validasi model
        results = model.val(data=temp_yaml_path, 
                split="test",
                imgsz=640,
                project=f"{project_base_few_shot}/validation",
                name=f"{shot}-shot-{size}",
                exist_ok=True)
        map50 = results.results_dict.get("metrics/mAP50(B)", 0.0)
        all_map50_scores[size][shot] = map50

        # Hapus folder runs agar bersih untuk next loop
        if os.path.exists("runs"):
            shutil.rmtree("runs")

# ======================================================
# Visualisasi Hasil
# ======================================================
plt.figure(figsize=(15, 8))
plt.title("Perbandingan mAP@0.5 Few-Shot per Model dan Ukuran Dataset Uji")
plt.xlabel("Jumlah Gambar per Kelas (Support-Set)")
plt.ylabel("mAP@0.5")
plt.xticks(shots)
plt.grid(True, linestyle='--', alpha=0.7)

# Plot setiap model (nano, small, medium)
for size in sizes:
    sorted_scores = sorted(all_map50_scores[size].items())
    x = [item[0] for item in sorted_scores]
    y = [item[1] for item in sorted_scores]
    plt.plot(x, y, marker='o', linestyle='-', label=f"{size} model")

plt.legend(title="Model", loc='lower right')
plt.tight_layout()
plt.show()

# ======================================================
# Cetak hasil lengkap
# ======================================================
print("\n--- Hasil Lengkap mAP@0.5 ---")
for size in sizes:
    print(f"Model {size}:")
    for shot, score in all_map50_scores[size].items():
        print(f"  - {shot}-shot diuji di {shot}-images: {score:.4f}")

In [ ]:
# import yaml
# import matplotlib.pyplot as plt
# from ultralytics import YOLO
# import os
# import shutil # Import shutil untuk operasi penghapusan folder

# # Define the models to be evaluated and the test datasets
# # Models trained with these 'shot' counts
# lists_model_shots = [5] + list(range(10, 51, 5))
# # Test datasets with these 'image' counts per class
# lists_test_dataset_images = [5] + list(range(10, 51, 5))

# # Dictionary to store mAP@0.5 scores for each model-test_dataset combination
# # Format: {model_shot_count: {test_dataset_image_count: mAP50_score}}
# all_map50_scores = {}

# # Path to your original data.yaml file
# data_yaml_original_path = "dataset/support_set/data.yaml"
# # **MODIFIKASI DI SINI:** Temporary directory to save modified data.yaml files
# # Sekarang akan disimpan di direktori yang sama dengan data_yaml_original_path
# temp_data_yaml_dir = os.path.dirname(data_yaml_original_path)
# # Default base directory for Ultralytics YOLO runs
# yolo_runs_dir = "runs" # This is the default folder where YOLO saves validation results

# # Ensure the temporary directory for YAMLs exists (though it should for support_set)
# os.makedirs(temp_data_yaml_dir, exist_ok=True)

# # Function to clear the runs/detect folder
# def clear_yolo_runs_directory(base_dir="."):
#     """
#     Menghapus folder 'runs/detect' dari direktori dasar yang diberikan.
#     """
#     detect_run_path = os.path.join(base_dir, yolo_runs_dir, 'detect')
#     if os.path.exists(detect_run_path):
#         print(f"Menghapus direktori: {detect_run_path}")
#         try:
#             shutil.rmtree(detect_run_path)
#             print(f"Direktori {detect_run_path} berhasil dihapus.")
#         except OSError as e:
#             print(f"Error: Gagal menghapus direktori {detect_run_path}. {e}")
#     else:
#         print(f"Direktori {detect_run_path} tidak ditemukan, tidak perlu dihapus.")

# # --- Start of Evaluation Loop ---
# # Loop through each trained model
# for model_shot_count in lists_model_shots:
#     print(f"\n--- Testing Model: {model_shot_count}-shot ---")

#     # Clear YOLO runs directory before validating each model
#     # This ensures a clean slate for each model's validation across test sets
#     clear_yolo_runs_directory()

#     # Initialize a dictionary to store scores for the current model
#     all_map50_scores[model_shot_count] = {}

#     # # Load the trained model
#     model_path = f"results/few_shot/training/{model_shot_count}-shot/weights/best.pt"
#     # try:
#     #     model = YOLO(model_path)
#     # except Exception as e:
#     #     print(f"Error loading model {model_path}: {e}. Skipping this model.")
#     #     continue # Skip to the next model if loading fails

#     # Loop through each test dataset
#     for test_image_count in lists_test_dataset_images:
#         model = YOLO(model_path)
#         # 1. Load the original data.yaml
#         with open(data_yaml_original_path, 'r') as f:
#             data_yaml = yaml.safe_load(f)

#         # 2. Update the 'test' path to the current test dataset
#         # This path is relative to the data.yaml itself.
#         # Ensure that 'dataset/support_set' is the base for 'X-images'
#         # For example, if data.yaml is in 'dataset/support_set/',
#         # and images are in 'dataset/support_set/5-images/',
#         # then the relative path from data.yaml to 5-images is '../5-images/images'
#         # This relative path needs to be correct based on your actual file structure.
#         data_yaml['test'] = f"../{test_image_count}-images/images"
#         data_yaml['train'] = f"../{test_image_count}-images/images"
#         data_yaml['val'] = f"../{test_image_count}-images/images"


#         # 3. Save the modified YAML to a temporary file in the specified directory
#         # **MODIFIKASI DI SINI:** temp_yaml_path sekarang menunjuk ke data_yaml_original_path's directory
#         temp_yaml_filename = f"data_support_test_{model_shot_count}model_{test_image_count}.yaml"
#         temp_yaml_path = os.path.join(temp_data_yaml_dir, temp_yaml_filename)
#         with open(temp_yaml_path, 'w') as f:
#             yaml.dump(data_yaml, f, sort_keys=False, default_flow_style=False)

#         # 4. Perform model validation
#         print(f"  Validating {model_shot_count}-shot model on {test_image_count}-images dataset...")
#         try:
#             # Pass the path to the newly created temporary YAML file
#             print("temp_yaml_path",temp_yaml_path)
#             results = model.val(data=temp_yaml_path)
#             # Get mAP@0.5 score
#             map50 = results.results_dict.get("metrics/mAP50(B)", 0.0)
#             all_map50_scores[model_shot_count][test_image_count] = map50
#             print(f"    mAP@0.5: {map50:.4f}")
#         except Exception as e:
#             print(f"Error validating {model_shot_count}-shot model on {test_image_count}-images: {e}")
#             all_map50_scores[model_shot_count][test_image_count] = 0.0 # Set to 0 if an error occurs

#         # Optional: Remove the temporary YAML file after use
#         # os.rmdir("runs/")
#         shutil.rmtree("runs/")

In [ ]:
# %matplotlib inline
# # --- Visualization of Results ---
# plt.figure(figsize=(15, 8))
# plt.title("Perbandingan mAP@0.5 Model Few-Shot pada Berbagai Ukuran Dataset Pengujian")
# plt.xlabel("Ukuran Dataset Pengujian (Jumlah Gambar per Kelas)")
# plt.ylabel("mAP@0.5")
# plt.xticks(lists_test_dataset_images)
# plt.grid(True, linestyle='--', alpha=0.7)

# # Plot a line for each trained model
# for model_shot_count, test_scores_dict in all_map50_scores.items():
#     # Sort the test scores by dataset size for proper plotting
#     sorted_test_scores = sorted(test_scores_dict.items())

#     # Extract test dataset image counts and corresponding mAP50 scores
#     test_dataset_sizes = [item[0] for item in sorted_test_scores]
#     map50_values = [item[1] for item in sorted_test_scores]

#     plt.plot(test_dataset_sizes, map50_values, marker='o', linestyle='-',
#              label=f"{model_shot_count}-shot Model")

# plt.legend(title="Model Dilatih (Shot)", loc='lower right')
# plt.tight_layout()
# plt.show()

# # Optional: Print the full results in a table-like format
# print("\n--- Complete mAP@0.5 Results ---")
# for model_shot_count, scores_by_dataset in all_map50_scores.items():
#     print(f"Model {model_shot_count}-shot:")
#     for test_img_count, score in scores_by_dataset.items():
#         print(f"  - Tested on {test_img_count}-images: {score:.4f}")

In [ ]:
if GoogleDrive:
    copy_to_google_drive("dataset/support_set", "dataset/support_set")
    copy_to_google_drive("results/support_set", "results/support_set")